# Vehicle Routing Problem (VRP) integrated with Google Maps Platform

### Libraries

In [1]:
import googlemaps
import gurobipy as gp
from gurobipy import GRB
import folium
import random
from googlemaps.convert import decode_polyline

### Create Matrix of Distances

In [2]:
def get_distance_matrix(addresses, api_key):
    """
    Uses the Google Distance Matrix API to build a 2D list of driving
    distances (in meters) between all pairs of addresses.
    """
    gmaps = googlemaps.Client(key=api_key)

    dm = gmaps.distance_matrix(
        origins=addresses,
        destinations=addresses,
        mode='driving',
        units='metric'
    )

    n = len(addresses)
    dist_matrix = [[0.0]*n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            elem = dm['rows'][i]['elements'][j]
            if elem['status'] == 'OK':
                dist_matrix[i][j] = float(elem['distance']['value'])
            else:
                # If no route is found
                dist_matrix[i][j] = float('inf')
    return dist_matrix

### VRP Model

In [3]:
def solve_vrp(dist_matrix, demands, vehicle_capacity, num_vehicles):
    """
    Solve a basic Capacitated VRP:
    - Single depot (node 0)
    - N-1 customers
    - 'num_vehicles' identical vehicles
    - Each vehicle has capacity = vehicle_capacity
    - Each customer must be visited exactly once

    Returns (routes, obj_val):
      routes: list of routes, routes[k] = the sequence of visited nodes for vehicle k
      obj_val: total distance of the solution
    """
    n = len(dist_matrix)
    depot = 0
    customers = range(1, n)
    K = num_vehicles

    m = gp.Model("CVRP")

    # x[k,i,j] = 1 if vehicle k travels from i to j
    x = m.addVars(K, n, n, vtype=GRB.BINARY, name="x")
    # u[k,i] = load on vehicle k upon arriving at node i
    u = m.addVars(K, n, lb=0, ub=vehicle_capacity, vtype=GRB.CONTINUOUS, name="u")

    # Objective: minimize total distance
    m.setObjective(
        gp.quicksum(dist_matrix[i][j] * x[k,i,j] for k in range(K) for i in range(n) for j in range(n)),
        GRB.MINIMIZE
    )

    # 1) Each customer is visited exactly once (sum over all vehicles, and all outgoing arcs):
    for i in customers:
        # Ensure that each customer i is visited exactly once by summing over all vehicles and possible departures.
        m.addConstr(
            gp.quicksum(x[k,i,j] for k in range(K) for j in range(n)) == 1,
            name=f"visit_{i}"
        )

    for k in range(K):
        # Ensure that each vehicle k leaves the depot exactly once (excluding self-loop from depot).
        m.addConstr(
            gp.quicksum(x[k,depot,j] for j in range(n) if j != 0) == 1,
            name=f"visit_{i}"
        )

    for k in range(K):
        # Ensure that each vehicle k returns to the depot exactly once (excluding self-loop to depot).
        m.addConstr(
            gp.quicksum(x[k,i,depot] for i in range(n) if i != 0) == 1,
            name=f"visit_{i}"
        )

    # 2) Flow conservation for each vehicle and each node (in = out):
    for k in range(K):
        for i in range(n):
            # Ensure the number of arcs entering node i equals the number leaving node i for vehicle k.
            m.addConstr(
                gp.quicksum(x[k,j,i] for j in range(n)) == gp.quicksum(x[k,i,j] for j in range(n)),
                name=f"flow_{k}_{i}"
            )

    # 3) Depot in/out must match for each vehicle if it’s used
    #    (the number of times a vehicle leaves the depot equals the number of times it returns).
    for k in range(K):
        # Ensure that for vehicle k, departures from the depot (to customers) equal returns to the depot.
        m.addConstr(
            gp.quicksum(x[k,depot,j] for j in customers) == gp.quicksum(x[k,i,depot] for i in customers),
            name=f"depot_inout_{k}"
        )

    # 4) Capacity constraints (MTZ):
    for k in range(K):
        # Start each vehicle k with load 0 at the depot.
        m.addConstr(u[k,depot] == 0, name=f"load_depot_{k}")

        for i in customers:
            # Check if the customer demand exceeds the vehicle capacity (feasibility check).
            if demands[i] > vehicle_capacity:
                print(f"Customer {i} demand = {demands[i]} exceeds vehicle capacity {vehicle_capacity}.")
                # This will likely be infeasible
            for j in customers:
                if i != j:
                    # MTZ constraint: ensure that the load carried is adjusted correctly when traveling from i to j.
                    m.addConstr(
                        u[k,i] - u[k,j] + vehicle_capacity * x[k,i,j] <= vehicle_capacity - demands[j],
                        name=f"cap_{k}_{i}_{j}"
                    )

    # No self-loops:
    for k in range(K):
        for i in range(n):
            # Prevent vehicle k from traveling from node i directly back to itself.
            m.addConstr(x[k,i,i] == 0, name=f"no_loop_{k}_{i}")

    for i in customers:
        for k in range(K):
            # Ensure that when a vehicle k visits customer i, the load is at least the demand of i.
            m.addConstr(u[k,i] >= demands[i])
                        
    m.optimize()

    if m.status == GRB.OPTIMAL:
        obj_val = m.objVal

        print("\nArc decisions x[k,i,j] in the optimal solution:")
        for k in range(K):
            for i in range(n):
                for j in range(n):
                    if x[k,i,j].X > 0.5:
                        print(f"x[{k}][{i}][{j}] = 1")

        # Reconstruct each vehicle’s route
        routes = []
        for k in range(K):
            route_k = [depot]
            current = depot
            while True:
                next_j = None
                # find j where x[k, current, j] == 1
                for j in range(n):
                    if x[k,current,j].X > 0.5:
                        next_j = j
                        break
                if next_j is None:
                    # no next move => route ended (vehicle not used or route finished)
                    break
                if next_j == depot:
                    # returned to depot => route closed
                    route_k.append(depot)
                    break
                else:
                    route_k.append(next_j)
                    current = next_j

            routes.append(route_k)

        return routes, obj_val
    else:
        print("No optimal solution found or model infeasible.")
        return None, None

### Latitude/longitude

In [4]:
def geocode_addresses(addresses, api_key):
    """
    Get latitude/longitude for each address using Google Geocoding API.
    Returns list of (lat, lng) in the same order as 'addresses'.
    """
    gmaps = googlemaps.Client(key=api_key)
    coords = []
    for addr in addresses:
        result = gmaps.geocode(addr)
        if result:
            loc = result[0]['geometry']['location']
            coords.append((loc['lat'], loc['lng']))
        else:
            coords.append((0.0, 0.0))
    return coords

### Plot

In [5]:
def plot_vrp_routes_on_map(addresses, coords, routes, api_key, outfile="vrp_route.html"):
    """
    Plots each vehicle's route with a different color on
    a Folium map using the road paths.
    """
    gmaps = googlemaps.Client(key=api_key)

    # Initialize map around the depot (node 0)
    start_lat, start_lng = coords[0]
    fmap = folium.Map(location=[start_lat, start_lng], zoom_start=13)

    # Some distinct route colors. Expand if you have many vehicles.
    route_colors = [
        "red", "blue", "green", "purple", "orange", "darkred", 
        "cadetblue", "darkblue", "darkgreen", "pink"
    ]
    # Optional
    dash_styles = [None, "5,5", "1,5", "5,10"]

    for k, route in enumerate(routes):
        color = route_colors[k % len(route_colors)]
        dash = dash_styles[k % len(dash_styles)] if dash_styles else None

        # Place markers for each stop
        for order, node_idx in enumerate(route):
            lat, lng = coords[node_idx]
            popup_text = f"Vehicle {k+1} - Stop {order+1}: {addresses[node_idx]}"
            # depot marker is different
            if node_idx == 0:
                folium.Marker([lat, lng], popup=popup_text,
                              icon=folium.Icon(color='blue', icon='info-sign')
                             ).add_to(fmap)
            else:
                folium.Marker([lat, lng], popup=popup_text,
                              icon=folium.Icon(color='green', icon='info-sign')
                             ).add_to(fmap)

        # Draw polylines for each pair in the route
        for i in range(len(route) - 1):
            origin_idx = route[i]
            dest_idx = route[i+1]
            origin_lat, origin_lng = coords[origin_idx]
            dest_lat, dest_lng = coords[dest_idx]

            directions_result = gmaps.directions(
                (origin_lat, origin_lng),
                (dest_lat, dest_lng),
                mode="driving"
            )
            if directions_result:
                polyline_str = directions_result[0]['overview_polyline']['points']
                polyline_points = decode_polyline(polyline_str)
                folium.PolyLine(
                    locations=[(pt['lat'], pt['lng']) for pt in polyline_points],
                    color=color,
                    weight=4,
                    opacity=1,
                    dash_array=dash  
                ).add_to(fmap)

    fmap.save(outfile)
    print(f"Map saved to {outfile}. Open 'vrp_route.html'.")

### Choose Locations

In [6]:
# 8 locations around Midtown Atlanta, with starting point at Georgia Tech
addresses = [
    "Georgia Tech, 225 North Ave NW, Atlanta, GA 30332",
    "Fox Theatre, 660 Peachtree St NE, Atlanta, GA 30308",
    "Atlanta Botanical Garden, 1345 Piedmont Ave NE, Atlanta, GA 30309",
    "Ponce City Market, 675 Ponce De Leon Ave NE, Atlanta, GA 30308",
    "Coca-Cola Headquarters, 1 Coca Cola Plz NW, Atlanta, GA 30313",
    "Mercedes-Benz Stadium, 1 AMB Dr NW, Atlanta, GA 30313",
    "Georgia Aquarium, 225 Baker St NW, Atlanta, GA 30313",
    "World of Coca-Cola, 121 Baker St NW, Atlanta, GA 30313"
]

### API Key

In [7]:
# Replace with your valid Google Cloud API key
api_key = "YOUR_API"

### Solve the Model

In [8]:
# 1. Build distance matrix
dist_matrix = get_distance_matrix(addresses, api_key)
n = len(dist_matrix)

# 2. Create demand
random.seed(42)
demands = [0] + [random.randint(1,3) for _ in range(n-1)]
print("Demands:", demands)

# 3. Solve VRP
vehicle_capacity = 4
num_vehicles = 3
routes, objVal = solve_vrp(dist_matrix, demands, vehicle_capacity, num_vehicles)

print(f"\nSolution found with total distance = {objVal:.2f} meters")
for k, r in enumerate(routes):
    print(f"Vehicle {k+1}: {r} (demands covered: {[demands[node] for node in r if node != 0]})")

# 4. Plot the route with real roads
coords = geocode_addresses(addresses, api_key)

# 5) Plot each vehicle’s route in a different color/style
plot_vrp_routes_on_map(addresses, coords, routes, api_key, outfile="vrp_route.html")


Demands: [0, 3, 1, 1, 3, 2, 1, 1]
Set parameter Username
Academic license - for non-commercial use only - expires 2026-03-13
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 214 rows, 216 columns and 1014 nonzeros
Model fingerprint: 0xe357e0b5
Variable types: 24 continuous, 192 integer (192 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [2e+02, 6e+03]
  Bounds range     [1e+00, 4e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 72 rows and 45 columns
Presolve time: 0.00s
Presolved: 142 rows, 171 columns, 1221 nonzeros
Variable types: 21 continuous, 150 integer (150 binary)

Root relaxation: objective 1.476900e+04, 72 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |    